In [ ]:
import numpy as np
import torch
import torch.optim as optim
import models
import utils
import matplotlib.pyplot as plt
import sklearn
from sklearn.manifold import TSNE
import os
from dataset import DataSet
import warnings

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
lcs = utils.get_data('./test_d2',start_col=1)

In [ ]:
utils.preview_lcs(lcs.data_obj['train_loader'], lcs.bands, N=10, figsize=(25,25))

In [ ]:
utils.preview_mask(lcs.data_obj['train_loader'], lcs.bands, N=10, figsize=(25,15))

In [ ]:
ds = DataSet(start_col=1)
ds.add_band('i', './ZTF_DR_data/i')
ds.to_numpy()
ds.set_intrinsic_vars()
ds.set_mean_mags()
ds.normalize()
# ds.set_epoch_counts(sep_std=3, plot=True, figsize=(10,5), index=15)
# ds.formatting()
# ds.set_union_tp()
# ds.set_data_obj(batch_size=8)



In [ ]:
plt.hist(ds.mean_mags)

In [ ]:
plt.hist(ds.intrinsic_vars)

In [ ]:
def chop_lcs(self, std_threshold=1):
    ranges = [np.ptp(lc[:,0]) for object_lcs in self.dataset for lc in object_lcs]
    std_ranges = np.std(ranges)
    for i, object_lcs in enumerate(self.dataset):
        for j, lc in enumerate(object_lcs):
            num_splits = int(ranges[i*j] / (std_threshold * std_ranges))
            if num_splits <= 1:
                continue
            else:
                split_pt = np.where(lc[:,0] > (std_threshold * std_ranges))[0][0]
                self.dataset[i][j] = lc[:split_pt]
            
            
def set_optical_lum(self):
        optical_lum = np.zeros((len(self.dataset),len(self.bands)))
        for i, object_lcs in enumerate(self.dataset):
            for j, lc in enumerate(object_lcs):
                dev_from_mean = (lc[:,1] - np.mean(lc[:,1]))
                avg_sq_dev_from_mean = np.matmul(dev_from_mean, dev_from_mean) * (1 / (len(lc) - 1))
                avg_sq_err = np.matmul(lc[:,2], lc[:,2]) / len(lc)
                intrinsic_var = avg_sq_dev_from_mean - avg_sq_err
                intrinsic_vars[i,j] = intrinsic_var
        self.intrinsic_vars = intrinsic_vars

In [ ]:
ZTF = utils.get_data(start_col=1, folder='./test_d1')
ZTF.set_target_x(num_points=400)
utils.preview_lcs(ZTF.data_obj['train_loader'], ZTF.bands, N=10, figsize=(25,25))

In [ ]:
# need to install astropy
from astropy.constants import G, c, h, k_B, M_sun, sigma_sb
from math import pi, exp
from scipy.integrate import dblquad
from scipy import signal
from scipy import signal 
from astropy import units as u

M_acc = M_sun / u.year
M_smbh = 2*(10**8)*M_sun
L_ratio = 0.1  # L / L_edd
eta = 0.1    # accretion efficiency
q = lambda lambd: (h * c) / (lambd * k_B)
x = lambda R, R_lambd: R / R_lambd
dirac_loc = lambda R, i, theta: (R/c) * (1 + np.sin(i) * np.cos(theta))
i = 0 # degrees

def radius(lambd): # wavelength should be in meters
    term1 = q(lambd) ** -(4/3)
    term2 = (((3 * G * M_acc*M_smbh) / (8 * pi * eta * c**2 * sigma_sb)) * L_ratio) ** 1/3
    R_lambd = term1 * term2
    return R_lambd
    
def dB_dT(lambd, R):
    R_lambd = radius(lambd)
    term1 = q(lambd) *  (x(R,R_lambd) ** -(1/4)) * exp(q(lambd) * (x(R,R_lambd) ** (3/4)))
    term2 = 4 * (exp(q(lambd) * x(R,R_lambd) **(3/4)) - 1) ** 2
    dB_dT = one / two
    return dB_dT

def dirac_fn(val, x):
    return 1 if val == x else 0

def transfer_fn(tau, lambd, R_in, R_out, D):
    res = dblquad(lambda R, theta:    \
                dB_dT(lambd, R) *    \
                ((np.cos(i) * R) / (D ** 2)) * \
                dirac_fn(tau, dirac_loc(R, i, theta)), \
                R_in, R_out, 0, 2*pi)
    return res
    
    
# questions?
# why does dB_dT return nans? is it the range?
# what's the units for wavelength?
# what are some reasonable input values for these equations



In [ ]:
##λ eff , u = 3671 Å, λ eff , g = 4827 Å, λ eff , r = 6223 Å, λ eff , i = 7546 Å, λ eff , z = 8691 Å, and λ eff , y = 9712 Å.
u = 3.671e-7 
g = 4.827e-7
r = 6.223e-7
i = 7.7546e-7
z = 8.8691e-7
y = 9.712e-7

    # u = 320.5 - 393.5, g = 401.5 - 551.9, r = 552.0 - 691.0, i = 691.0 - 818.0,
    # z = 818.0 - 923.5, y = 923.8 - 1084.5


